# Proyecto de Web Scrapping del Marketplace Yapo.

1. Scrapping de Datos - Yapo

In [1]:
#Importamos las librerias necesarias
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
import time
import requests
import pandas as pd
from datetime import datetime, timedelta
import seaborn as sns
import scipy as sp
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

#Automcompletar rápido
%config IPCompleter.greedy=True

#Display 6 registros
pd.options.display.min_rows = 6

Con la libreria bs4, abriremos un navegador Firefox, en la pagina de los anuncios de Yapo

In [12]:
browser = webdriver.Firefox() #Cargamos el driver a una variable
myUrl = "https://www.yapo.cl/region-metropolitana/marketplace/computadores-electronica/celulares?orden=date-desc&buscar=xiaomi&pagina="
browser.get(myUrl) #Abrimos Firefox y entramos a la pagina sin loguearnos.
pageSoup = soup(browser.page_source, "html.parser")

KeyboardInterrupt: 

In [3]:
def webscraping(numero):
    pagina = "https://www.yapo.cl/region-metropolitana/marketplace/computadores-electronica/celulares?orden=date-desc&buscar=xiaomi&pagina="
    pag = pagina+str(numero)
    browser.get(pag) #Abrimos Chrome y entramos a la pagina sin loguearnos.
    time.sleep(3)
    pageSoup = soup(browser.page_source, "html.parser")

    #Informacion
    #titulo    
    titulo_h2 = pageSoup.find_all("h2", class_="title m-0 text-truncate pb-2")

    #precio
    precios_span = pageSoup.find_all("span", class_="clp-price d-flex display-price ng-star-inserted")

        #fecha de publicacion
    fecha_p = pageSoup.find_all("p", class_="d-flex m-0 created col ng-star-inserted")

    #hora de publicacion
    hora_p = pageSoup.find_all("p", class_="d-flex m-0 created col ng-star-inserted")

    #lugar
    lugar_span = pageSoup.find_all("span", class_="label align-self-center")

    #dataframes
    df_2 = pd.DataFrame(0, index=range(50), columns=["titulo","precio","fecha_publicacion", "hora_publicacion", "lugar_venta"])
    #Reiniciamos el index para quedar con rangeindex y no con un multiindex
    df_2.reset_index(drop=True, inplace=True)


    #Poblamos titulo.
    df_2["titulo"][:len(titulo_h2)] = titulo_h2
    df_2["titulo"] = df_2["titulo"].astype(str)
    df_2["titulo"]= df_2["titulo"].str.replace('<h2 _ngcontent-serverapp-c2620551781="" class="title m-0 text-truncate pb-2">', '').str.replace('</h2>', '')
    df_2["titulo"]= df_2["titulo"].str.replace('!', '').str.replace('¡', '')

    #Poblamos precio
    df_2["precio"][:len(precios_span)] = precios_span
    df_2["precio"] = df_2["precio"].astype(str)
    df_2["precio"] = (df_2["precio"].str.replace(
            '<span _ngcontent-serverapp-c2620551781="" class="clp-price d-flex display-price ng-star-inserted"><span _ngcontent-serverapp-c2620551781="" class="bracket">(</span>$', '').str.replace(
                '<span _ngcontent-serverapp-c2620551781="" class="bracket">)</span></span>', '').str.replace('.',''))

    #Poblamos la fecha
    df_2["fecha_publicacion"][:len(fecha_p)] = fecha_p
    df_2["fecha_publicacion"]= df_2["fecha_publicacion"].astype(str)
    df_2["fecha_publicacion"] = df_2["fecha_publicacion"].str.replace('<p _ngcontent-serverapp-c2620551781="" class="d-flex m-0 created col ng-star-inserted" title="', '')

    #iteramos para extraer solo la fecha.
    for indice, elemento in enumerate(df_2["fecha_publicacion"]):
        df_2.loc[indice, "fecha_publicacion"] = df_2.loc[indice, "fecha_publicacion"][:10]


    #Poblamos la hora
    df_2["hora_publicacion"][:len(hora_p)] = hora_p
    df_2["hora_publicacion"]= df_2["hora_publicacion"].astype(str)
    df_2["hora_publicacion"] = df_2["hora_publicacion"].str.replace('<p _ngcontent-serverapp-c2620551781="" class="d-flex m-0 created col ng-star-inserted" title="', '')


    #iteramos para extraer solo la hora.
    for indice, elemento in enumerate(df_2["hora_publicacion"]):
        df_2.loc[indice, "hora_publicacion"] = df_2.loc[indice, "hora_publicacion"][12:20]


    #Poblamos el lugar desde donde se publica el producto
    df_2["lugar_venta"][:len(lugar_span)] = lugar_span
    df_2["lugar_venta"] = df_2["lugar_venta"].astype(str)
    df_2["lugar_venta"] = df_2["lugar_venta"].str.replace('<span _ngcontent-serverapp-c2620551781="" class="label align-self-center">', '').str.replace(" <!-- --><!-- -->", "").str.replace("</span>", "")
    
    df_2.to_csv(fr"C:\Users\miche\OneDrive\Escritorio\Web Scrapping\04_Xiaomi\WS Datasets_Xiaomi\ws_xiaomi_pagina_{numero}.csv", index=False)


In [4]:
for i in range(1,26):
    if i == 25:
        webscraping(i)
        print(f"Proceso terminado: Ultima pagina {i}")
        break
    else:
        webscraping(i)

C:\Users\miche\AppData\Local\Temp\ipykernel_11584\3718078151.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2["precio"][:len(precios_span)] = precios_span
c:\Users\miche\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\cast.py:976: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)
C:\Users\miche\AppData\Local\Temp\ipykernel_11584\3718078151.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

Proceso terminado: Ultima pagina 25


C:\Users\miche\AppData\Local\Temp\ipykernel_11584\3718078151.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2["precio"][:len(precios_span)] = precios_span
c:\Users\miche\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\cast.py:976: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)
C:\Users\miche\AppData\Local\Temp\ipykernel_11584\3718078151.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

In [5]:
#Creamos un nuevo DataFrame
df_final = pd.DataFrame(0, index=range(1), columns=["titulo","precio","fecha_publicacion", "hora_publicacion", "lugar_venta"])

#Poblamos el nuevo DataFrame
for i in range(1,26):
    df_nuevo = pd.read_csv(fr"C:\Users\miche\OneDrive\Escritorio\Web Scrapping\04_Xiaomi\WS Datasets_Xiaomi\ws_xiaomi_pagina_{i}.csv")
    df_final = pd.concat([df_final ,df_nuevo],ignore_index=True)
      

In [6]:
#Con Nulos y duplicados - 43901 datos
df_final

,titulo,precio,fecha_publicacion,hora_publicacion,lugar_venta
0,0,0,0,0,0
1,Xiaomi Redmi Note 10 Pro 6/128 GB,80000,24-08-2023,19:53:59,"El Bosque, Región Metropolitana"
2,Xiaomi mi watch,90000,24-08-2023,18:40:03,"Puente Alto, Región Metropolitana"
...,...,...,...,...,...
1248,0,0,0,NaN,0
1249,0,0,0,NaN,0
1250,0,0,0,NaN,0


In [7]:
#Borramos los nulos
df_final.dropna(inplace=True)

In [8]:
#Borramos todas las columnas que tenga 0 en precio
df_final = df_final[df_final["titulo"] != 0]
df_final = df_final[df_final["precio"] != 0]

In [9]:
#Reiniciamos el Dataframe
df_final.reset_index(drop=True, inplace=True)

In [10]:
#sin nulos, duplicados o titulo = 0 - 41244 datos
df_final

,titulo,precio,fecha_publicacion,hora_publicacion,lugar_venta
0,Xiaomi Redmi Note 10 Pro 6/128 GB,80000,24-08-2023,19:53:59,"El Bosque, Región Metropolitana"
1,Xiaomi mi watch,90000,24-08-2023,18:40:03,"Puente Alto, Región Metropolitana"
2,Xiaomi Smart Watch S1 Active White,105000,24-08-2023,12:45:57,"La Florida, Región Metropolitana"
...,...,...,...,...,...
1078,Xiaomi redmi 10a 64 gb,50000,31-07-2022,19:14:06,"Lo Prado, Región Metropolitana"
1079,Xiaomi Redmi Note 10 S,95000,31-07-2022,12:51:51,"La Florida, Región Metropolitana"
1080,Celular Xiaomi Redmi note 8,90000,30-07-2022,17:11:22,"Ñuñoa, Región Metropolitana"


In [11]:
#Mostramos el Dataframe final
df_final.to_csv(r"C:\Users\miche\OneDrive\Escritorio\Web Scrapping\04_Xiaomi\df_xiaomi.csv", index=False)